In [2]:
import pickle
import gensim
from nltk.corpus import stopwords 
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet

In [3]:
path_to_mallet_binary = "Mallet/bin/mallet"

In [4]:
f = open("Processed_Docs.pkl", "rb")
processed_docs = pickle.load(f)

In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [7]:
dictionary.filter_extremes(no_below=15, no_above=0.5)

In [8]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [9]:
model = LdaMallet(path_to_mallet_binary, corpus=bow_corpus, num_topics=20, id2word=dictionary)

In [10]:
lda_model = model

In [11]:
lda_model.save("lda_Model_mallet_topics_20")

In [12]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5476621675675115


In [15]:
def convertldaMalletToldaGen(mallet_model):
    model_gensim = gensim.models.LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) 
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

In [16]:
lda_model = convertldaMalletToldaGen(lda_model)

In [18]:
import pyLDAvis
import pyLDAvis.gensim


In [19]:
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, 
                              corpus=bow_corpus, 
                              dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)